In [1]:
import re
import time
import json
import math
import loguru
import random
import requests
import selenium.webdriver.support.ui as ui
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from argparse import Namespace

/home/jasonhaven/anaconda3/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
import pymongo

In [3]:
class User(object):
    
    _zh_to_en={
                '昵称':'screen_name',
                '性别': 'gender',
                '地区': 'location',  
                '生日': 'birthday',
                '感情状况': 'emotion',
                '简介': 'description',
                '认证':'validation'
             }

    def __init__(self,data):
        for k,v in data.items():
            setattr(self,k,v)

    def __str__(self):
        dct={}
        for  attr in dir(self):
            if not attr.startswith('_'):
                dct[attr]=getattr(self,attr)
        return json.dumps(dct,ensure_ascii=False)

### 相关参数

In [4]:
args_login=Namespace(
    user_name='18392161976@163.com',
    password='18392161976',
)

logger = loguru.logger

headers = {
    'User-agent' : 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36',
    'Host' : 'm.weibo.cn',
    'Accept' : 'application/json, text/plain, */*',
    'Accept-Language' : 'zh-CN,zh;q=0.8,en-US;q=0.5,en;q=0.3',
    'Accept-Encoding' : 'gzip, deflate, br',
    'Referer' : 'https://m.weibo.cn',
    'Cookie' : '_T_WM=2809dae9dfad0eb937f1e966bb58baad; SCF=AqEMR7DjEq8IoKbjhAVfKBVnwlb_liwr_1DYNDiBOupIQYnwbDkA3-biaFeV6QYiEA3ERIAGA6NUZq8ijcg5qjQ.; MLOGIN=1; SUB=_2A25xvf-jDeRhGeFO71YT8SbEyj6IHXVTQYHrrDV6PUJbkdAKLRjYkW1NQX7CgVC1B56oykRpTufwN26nl_WiFELK; SUHB=09Pt0V0gyhebms; SSOLoginState=1555664883; M_WEIBOCN_PARAMS=luicode%3D20000174',
    'Connection' : 'keep-alive',
}


url_weibocn_profile='https://weibo.cn/{}/info'
url_mweibo_profile="https://m.weibo.cn/profile/info?uid={}"
url_mweibo="https://m.weibo.cn"
url_mweibo_posts="https://m.weibo.cn/api/container/getIndex?containerid={}&page={}"
url_mweibo_fans="https://m.weibo.cn/api/container/getIndex?containerid=231051_-_fans_-_{}&since_id={}"
url_mweibo_following="https://m.weibo.cn/api/container/getIndex?containerid=231051_-_followers_-_{}&page={}"
url_mweibo_comments="https://m.weibo.cn/comments/hotflow?id={}&mid={}&max_id={}&max_id_type=0"
url_mweibo_like_users='https://m.weibo.cn/api/statuses/repostTimeline?id={}&page={}'
url_mweibo_report_users='https://m.weibo.cn/api/attitudes/show?id={}&page={}'

### 登录获取dirver

In [5]:
def get_driver():
    options = Options()
    options.add_argument('-headless')
    return webdriver.Chrome(options=options)

def login():
    '''
    启动并登录
    '''
    url_login="https://passport.weibo.cn/signin/login"
    driver=get_driver()
    wait=ui.WebDriverWait(driver,10)
    driver.get(url_login)
    time.sleep(2)#等待跳转
    
    try:
        driver.find_element_by_class_name('txt-shadow')
    except:
        elem_user=driver.find_element_by_id("loginName")
        elem_pwd=driver.find_element_by_id("loginPassword")
        elem_user.send_keys(args_login.user_name)
        elem_pwd.send_keys(args_login.password)
        try:
            elem_sub=driver.find_element_by_id("loginAction")
            elem_sub.click()
            time.sleep(3)#等待验证显示
            elem_sub=driver.find_element_by_id("embed-captcha")
            elem_sub.click()
            time.sleep(3)#等待跳转
        except:
            logger.info('login success!')
            return driver
    logger.info('login success!')
    return driver

In [6]:
driver = login()

2019-04-29 19:25:36.608 | INFO     | __main__:login:31 - login success!


### 获取用户的基本信息

In [7]:
def download_html(url,retry=3):
    try:
        proxy=random.choice([{'http':'//'.join(['http:',l.strip()])}for l in open('./ProxyGetter/proxies.txt').readlines()])
        resp=requests.get(url = url,headers = headers,proxies=proxy)
        if resp.status_code != 200:
            logger.error('url open error: {}'.format(url))
        return resp.text
    except Exception as e:
        logger.debug("failed and retry: {}".format(url))
        if retry > 1:
            time.sleep(1)
            retry -= 1
            return download_html(url,retry)
        
def get_user_info_from_mweibo(uid):
    text=download_html(url_mweibo_profile.format(uid))
    data=json.loads(text)
    data=data['data']['user']
    infobox={}
    infobox['_id']=data['id']
    infobox['screen_name']=data['screen_name'].strip()
    infobox['statuses_count']=data['statuses_count']
    infobox['followers_count']=data['followers_count']
    infobox['follow_count']=data['follow_count']
    infobox['description']=data['description'].strip()
    infobox['gender']='男' if data['gender']=='m' else '女'
    
    infobox['verified']=data['verified']
    infobox['verified_type']=data['description']
    infobox['verified_reason']=data['description'].strip()
    return infobox

def get_user_info_from_weibocn(uid):
    driver.get(url_weibocn_profile.format(uid))
    time.sleep(2)#等待跳转
    text=driver.find_element_by_xpath("//div[text()='基本信息']/following-sibling::div[1]").text
    data=dict([re.split(r':|：',s) for s in text.split('\n') ])
    infobox={}
    for k,v in data.items():
        if k not in User._zh_to_en:
            infobox['other']=v
        else:
            infobox[User._zh_to_en[k]]=v
    return infobox

def merge_user_info(mweibo_dct,weibocn_dct):
    for k,v in weibocn_dct.items():
        if k not in mweibo_dct:
            mweibo_dct[k]=v
    return mweibo_dct

def get_user_info(uid):
    try:
        d1=get_user_info_from_weibocn(uid)
        d2=get_user_info_from_mweibo(uid)
        d=merge_user_info(d1,d2)
        return d
    except Exception as e:
        logger.debug(e)
        return get_user_info_from_mweibo(uid)

In [8]:
u=get_user_info(uid='2481058012')

In [10]:
import pymongo

client = pymongo.MongoClient("mongodb://localhost:27017/")
weibo_db = client["weibo"]
user_table = weibo_db['user']

for t in user_table.find():
    u = get_user_info(uid=t['_id'])
    user_table.update({'_id':t['_id']},u)
    print('update success id = {}'.format(u['_id']))
    time.sleep(2)

/home/jasonhaven/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  if __name__ == '__main__':


update success id = 7044218812
update success id = 2481058012
update success id = 1740577714
update success id = 1752502537


2019-04-29 19:29:48.780 | DEBUG    | __main__:get_user_info:59 - dictionary update sequence element #6 has length 4; 2 is required


update success id = 2028810631
update success id = 1784473157


2019-04-29 19:29:57.868 | DEBUG    | __main__:get_user_info:59 - dictionary update sequence element #6 has length 3; 2 is required


...

update success id = 1246850033


2019-04-29 19:44:20.576 | DEBUG    | __main__:get_user_info:59 - dictionary update sequence element #6 has length 3; 2 is required


update success id = 5255814135


2019-04-29 19:44:25.040 | DEBUG    | __main__:get_user_info:59 - dictionary update sequence element #6 has length 3; 2 is required


update success id = 1776448504
update success id = 5909342713


### 获取用户的关注列表,粉丝列表

In [45]:

def parse_url_friends(url):
	result = []
	text = download_html(url)
	dct = json.loads(text)
	for card in dct['data']['cards']:
		for group in card['card_group']:
			infobox = {}
			if 'desc1' in group:
				infobox['desc1'] = group['desc1']
			if 'desc2' in group:
				infobox['desc2'] = group['desc2']
			data = group['user']
			infobox['_id'] = data['id']
			infobox['screen_name'] = data['screen_name'].strip()
			try:
				infobox['statuses_count'] = data['statuses_count']
				infobox['followers_count'] = data['followers_count']
				infobox['follow_count'] = data['follow_count']
				infobox['description'] = data['description'].strip()
				infobox['gender'] = '男' if data['gender'] == 'm' else '女'
				infobox['verified'] = data['verified']
				infobox['verified_type'] = data['description']
				infobox['verified_reason'] = data['description'].strip()
			except:
				logger.error(card['user'])
			result.append(infobox)
	return result


def parse_first_page(url, flag):
	'''
	flag: '全部关注','全部粉丝'
	'''
	result = []
	text = download_html(url)
	dct = json.loads(text)
	for card in dct['data']['cards']:
		if 'title' in card and card['title'].endswith(flag):
			for group in card['card_group']:
				infobox = {}
				if 'desc1' in group:
					infobox['desc1'] = group['desc1']
				if 'desc2' in group:
					infobox['desc2'] = group['desc2']
				data = group['user']
				infobox['_id'] = data['id']
				infobox['screen_name'] = data['screen_name'].strip()
				try:
					infobox['statuses_count'] = data['statuses_count']
					infobox['followers_count'] = data['followers_count']
					infobox['follow_count'] = data['follow_count']
					infobox['description'] = data['description'].strip()
					infobox['gender'] = '男' if data['gender'] == 'm' else '女'
					infobox['verified'] = data['verified']
					infobox['verified_type'] = data['description']
					infobox['verified_reason'] = data['description'].strip()
				except:
					logger.error(card['user'])
				result.append(infobox)
	return result
#关注列表
def get_all_followings(uid, follow_count=0):
	result = []
	text = download_html(url_mweibo_profile.format(uid))
	dct = json.loads(text)
	page_num = math.ceil(follow_count / 20)
	for i in range(page_num):
		url_following = url_mweibo_following.format(uid, i + 1)
		logger.info(url_following)

		if i == 0:
			followings = parse_first_page(url_following, '全部关注')
		else:
			followings = parse_url_friends(url_following)
		print(len(followings))
		result.extend(followings)
	return result


# 粉丝列表
def get_all_fans(uid, follow_count=0):
	result = []
	page_num = math.ceil(follow_count / 20)
	for i in range(page_num):
		time.sleep(random.random() * 10)
		url_follower = url_mweibo_fans.format(uid, i + 1)
		logger.info(url_follower)

		if i == 0:
			followings = parse_first_page(url_follower, '全部粉丝')
		else:
			followings = parse_url_friends(url_follower)
		print(len(followings))
		result.extend(followings)
	return result

In [41]:
# rst = get_all_followings('1887344341',50)

In [ ]:
get_all_fans('1642591402',100)

### 获取用户的全部微博

In [48]:
def get_user_posts(uid,statuses_count=0):
    result=[]
    text=download_html(url_mweibo_profile.format(uid))
    dct=json.loads(text)
    page_num=math.ceil(statuses_count/10)
    for i in range(page_num):
        url_posts=url_mweibo_posts.format(dct['data']['more'][3:],i+1)
        logger.info(url_posts)
        posts=parse_url_posts(url_posts)
        print(len(posts))
        result.extend(posts)
    return result

def parse_url_posts(url_posts):
    result=[]
    text=download_html(url_posts)
    data=json.loads(text)
    for card in data['data']['cards'][1:]:
        dct={}
        if 'mblog' not in card:
            logger.error('no "mblog" in card! {}'.format(url_posts))
            break
        dct['_id']=int(card['mblog']['id'])
        dct['created_at']=card['mblog']['created_at']
        dct['text']=card['mblog']['text']
        dct['source']=card['mblog']['source']
        dct['reposts_count']=card['mblog']['reposts_count']
        dct['comments_count']=card['mblog']['comments_count']
        dct['attitudes_count']=card['mblog']['attitudes_count']
        dct['isLongText']=card['mblog']['isLongText']
        
        if 'page_info' in card['mblog']: 
            dct['page_info']=card['mblog']['page_info']
        result.append(dct)
    return result

In [ ]:
posts = get_user_posts('1642591402',20) 

### 获取微博的点赞列表和转发列表

In [6]:
def get_weibo_likes(pid,attitudes_count=50):
    result=[]
    page_num=math.ceil(attitudes_count/50)
    for i in range(page_num):
        url=url_mweibo_like_users.format(pid,i+1)
        logger.info(url)
        followings=parse_url_likes(url)
        result.extend(followings)
    return result

def get_weibo_reports(pid,reposts_count=10):
    result=[]
    page_num=math.ceil(reposts_count/50)
    for i in range(page_num):
        url=url_mweibo_report_users.format(pid,i+1)
        logger.info(url)
        followings=parse_url_reports(url)
        result.extend(followings)
    return result

def parse_url_likes(url):
    result=[]
    text=download_html(url)
    data=json.loads(text)
    for card in data['data']['data']:
        infobox={}
        infobox['id']=card['user']['id']
        infobox['screen_name']=card['user']['screen_name'].strip()
        try:
            infobox['statuses_count']=card['user']['statuses_count']
            infobox['followers_count']=card['user']['followers_count']
            infobox['follow_count']=card['user']['follow_count']
            infobox['description']=card['user']['description'].strip()
            infobox['gender']='男' if card['user']['gender']=='m' else '女'
        except:
            logger.error(card['user'])
        result.append(infobox)
    return result

def parse_url_reports(url):
    result=[]
    text=download_html(url)
    data=json.loads(text)
    for card in data['data']['data']:
        infobox={}
        infobox['_id']=int(card['user']['id'])
        infobox['screen_name']=card['user']['screen_name'].strip()
        try:
            infobox['statuses_count']=card['user']['statuses_count']
            infobox['followers_count']=card['user']['followers_count']
            infobox['follow_count']=card['user']['follow_count']
            infobox['description']=card['user']['description'].strip()
            infobox['gender']='男' if card['user']['gender']=='m' else '女'
            infobox['verified']=card['user']['verified']
            infobox['verified_type']=card['user']['description']
            infobox['verified_reason']=card['user']['description'].strip()
        except:
            logger.error(card['user'])
        result.append(infobox)
    return result

In [67]:
# get_weibo_likes(4363904852823605)

In [ ]:
get_weibo_reports(4363904852823605)

### 获取微博的全部评论

In [22]:
def get_all_comments(pid,max_num=10):
    result=[]
    url="https://m.weibo.cn/comments/hotflow?id={}&mid={}&max_id_type=0".format(pid,pid)
    logger.info(url)
    text=download_html(url)
    dct=json.loads(text)
    total_num=dct['data']['total_number']
    logger.info('total comments:{}'.format(total_num))
    error_count=5
    while total_num>0 and len(result)<max_num and error_count>0:
        logger.info(url)
        text=download_html(url)
        dct=json.loads(text)
        comments=parse_comments(pid,dct)
        if comments == []:
            error_count -=1
            logger.error('comments == []')
            continue
        error_count=10
        result.extend(comments)
        max_id=dct['data']['max_id']
        url=url_mweibo_comments.format(pid,pid,max_id)
    return result

def parse_comments(pid, data):
    result = []
    if 'data' not in data or 'data' not in data['data']:
        logger.error('key not exists!')
        return result
    for comment in data['data']['data']:
        dct={}
        dct['pid']=pid
        dct['_id']=comment['id']
        dct['created_at']=comment['created_at']
        dct['text']=comment['text']
        dct['uid']=comment['user']['id']
        dct['like_count']=comment['like_count']
        dct['comments']=[]
        try:
            if isinstance(comment['comments'], list):
                dct['comments'] = parse_comments_of_comment(comment['comments'])
        except Exception as e:
            logger.error('parse comment failed!')
            print(e)
        result.append(dct)
    return result

def parse_comments_of_comment(comments:list):
    result=[]
    for card in comments:
        dct={}
        dct['created_at']=card['created_at']
        dct['text']=card['text']
        dct['uid']=card['user']['id']
        #递归评论
        #if card['comments']!=False:
        #    dct['comments']=parse_comments_of_comment(card['comments'])
        result.append(dct)
    return result

In [23]:
get_all_comments(pid=4365293364446663,max_num=10)

2019-04-26 12:18:19.864 | INFO     | __main__:get_all_comments:4 - https://m.weibo.cn/comments/hotflow?id=4365293364446663&mid=4365293364446663&max_id_type=0
2019-04-26 12:18:20.073 | INFO     | __main__:get_all_comments:8 - total comments:17
2019-04-26 12:18:20.074 | INFO     | __main__:get_all_comments:11 - https://m.weibo.cn/comments/hotflow?id=4365293364446663&mid=4365293364446663&max_id_type=0


[{'pid': 4365293364446663,
  '_id': '4365295471152492',
  'created_at': 'Fri Apr 26 11:56:45 +0800 2019',
  'text': '没文化的想问一句，查重为什么要收费啊……这点太特么的操蛋了',
  'uid': 1835796702,
  'like_count': 2,
  'comments': [{'created_at': 'Fri Apr 26 12:04:44 +0800 2019',
    'text': '查重不贵的，而且抄没抄自己心里还没数吗<span class="url-icon"><img alt=[允悲] src="//h5.sinaimg.cn/m/emoticon/icon/default/d_yunbei-9aa3c436a4.png" style="width:1em; height:1em;" /></span>',
    'uid': 5925269664},
   {'created_at': 'Fri Apr 26 12:09:25 +0800 2019',
    'text': "回复<a href='/n/iClaude'>@iClaude</a>:哈哈哈～～后面那句直接说中重点～～我主要是觉得，应该来讲是个工具。如果是在校学生，还是应该要免费使用",
    'uid': 1835796702}]},
 {'pid': 4365293364446663,
  '_id': '4365295730758190',
  'created_at': 'Fri Apr 26 11:57:47 +0800 2019',
  'text': '虽然查重率降低是很烦，但是学术要求严格难道不是好事吗？所以不能混日子了才这么愤怒？',
  'uid': 6616845159,
  'like_count': 2,
  'comments': [{'created_at': 'Fri Apr 26 12:05:02 +0800 2019',
    'text': '看原博评论，应该是从标准到手续都更麻烦了。查重要求的重复率也调低了，说实话，很多时候知网查重算法让人很没办法的，有些话引用了也会标出来，如果引用了一些数据报告，或者领

In [63]:
comments[0]

{'pid': 4363242803158517,
 '_id': '4363243281804679',
 'created_at': 'Sat Apr 20 20:02:06 +0800 2019',
 'text': '<a href=\'/n/新浪娱乐\'>@新浪娱乐</a> 李易峰的腿好长啊<span class="url-icon"><img alt=[跪了] src="//h5.sinaimg.cn/m/emoticon/icon/default/d_guile-7b3e474f7f.png" style="width:1em; height:1em;" /></span><span class="url-icon"><img alt=[跪了] src="//h5.sinaimg.cn/m/emoticon/icon/default/d_guile-7b3e474f7f.png" style="width:1em; height:1em;" /></span>，美貌帅气，身材也超级好，气质也非常好，能打包带走藏起来一个人看吗<span class="url-icon"><img alt=[跪了] src="//h5.sinaimg.cn/m/emoticon/icon/default/d_guile-7b3e474f7f.png" style="width:1em; height:1em;" /></span><span class="url-icon"><img alt=[跪了] src="//h5.sinaimg.cn/m/emoticon/icon/default/d_guile-7b3e474f7f.png" style="width:1em; height:1em;" /></span><span class="url-icon"><img alt=[跪了] src="//h5.sinaimg.cn/m/emoticon/icon/default/d_guile-7b3e474f7f.png" style="width:1em; height:1em;" /></span>',
 'uid': 5622712244}

In [ ]:
if __name__ == "__main__":
    #driver = login()
    info=get_user_info(uid='7044218812')
    print(info)
    posts=get_user_posts('7044218812',statuses_count=10)
    print(len(posts))    
    followings=get_all_followings('7044218812',follow_count=1)
    print(len(followings))
    followers=get_all_fans('7044218812',followers_count=info.followers_count)
    print(followers)
    comments=get_all_comments(pid=posts[0]['id'])
    print(comments)